# Text Feature Extraction Homework

In this exercise, we will implement another slight variation of the `tfidf`  document distance definition using **sublinear** document counts.

We will then compare it to the `sklearn` implementation.

## Preliminaries

#### Imports

In [1]:
import string
import os
import pickle
from collections import Counter 

import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

import sys 
sys.path.append("../..")
from E4525_ML import text # you must have saved the file text.py into the E4525_ML directory

#### Data Directories

In [2]:
raw_data_dir=r"../../raw/C50/C50train" # original data set used for training
data_dir    =r"../../data/C50/"  # directory to save intermediate results

#### Convenience Function Definitions

A few functions carried over from the Text_Features notebook that we will need during this exercise.

In [3]:
def process_text(filename,stop): 
    porter_stemmer = PorterStemmer()
    file=open(filename)
    lines=file.readlines()
    text_str=" ".join(lines).replace("\n"," ").lower()
    stem_list=text.stem_tokenizer(text_str)
    used_list=[token for token in stem_list if token not in stop]
    return used_list

In [4]:
def text_2_set(filename,stop_words):
    stems=process_text(filename,stop_words)
    return set(stems)

In [5]:
def text_2_counts(filename,stop_words):
    stems=process_text(filename,stop_words)
    return Counter(stems)

In [6]:
def corpus_word_counts(documents,stop):
    counts=Counter()
    for filename in documents["filename"]:   
        print("processing...",filename)
        bag=text_2_set(filename,stop)
        for word in bag:
            counts[word]+=1
    return pd.DataFrame.from_dict(counts,orient="index")

## Environment Preparation

<div class="alert alert-block alert-info"> Problem 0 </div>

1. Download the  [Reuters 50](https://archive.ics.uci.edu/ml/datasets/Reuter_50_50) collection of texts. Save it on the `raw` data directory.

    You should end up with this directory structure structure:
    
    raw/
        C50/
            C50train/
            C50test/
            
1. Run to completion the [Text Feature Extraction](./Text_Features.ipynb) notebook. This will generate the document lists, and word count statistics. Make sure to run any of the sections are are meant to be run only once.
1. Save the text.py python module into the `E4525_ML` directory.

## Implement TF-IDF document Distance with Sublinear Growth 

<div class="alert alert-block alert-info"> Problem 1.1 </div>

Read the list of documents in the file `C50_documents.csv`  from the data directory `data_dir` into a `documents` variable

In [7]:
documents_filename=data_dir+"/C50_documents.csv"

documents=pd.read_csv(documents_filename,index_col="document_id")
documents.head()

,filename,label
document_id,,
0,../../raw/C50/C50train/ScottHillis/253868newsM...,ScottHillis
1,../../raw/C50/C50train/ScottHillis/305692newsM...,ScottHillis
2,../../raw/C50/C50train/ScottHillis/340736newsM...,ScottHillis
3,../../raw/C50/C50train/ScottHillis/140340newsM...,ScottHillis
4,../../raw/C50/C50train/ScottHillis/126593newsM...,ScottHillis


<div class="alert alert-block alert-info"> Problem 1.2 </div>

Create a list of stop works by calling the function `text.stop_words` from the `E4525.text` python module.

In [8]:
stop_words=text.stop_words()
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

<div class="alert alert-block alert-info"> Problem 1.3 </div>

Using pandas, read  the word count (term frequencies) file generated by the Text_Features notebook
The file is called "corpus_word_counts.csv"

In [9]:
word_counts_filename=data_dir+"corpus_word_counts.csv"

word_counts=pd.read_csv(word_counts_filename,index_col="word")
word_counts.head()

,count
word,
batteri,11
raytheon,7
intimid,9
commentari,20
issu,689


<div class="alert alert-block alert-info"> Problem 1.4 </div>
Create a variable $V$ with the vocabulary size  and a variable named $C$ with the total number of documents

In [10]:
# Vocabulary size
V=len(word_counts)
print("Vocabulary Size",V)
# Corpus size
C=len(documents)
print("Corpus Size",C)

Vocabulary Size 28060
Corpus Size 2500


<div class="alert alert-block alert-info"> Problem 1.5 </div>
Compute the smoothed inverse document counts, defined as
$$
    \textrm{idf}_i =  \log\left( \frac{1+C}{1+\textrm{n}_i}\right) + 1
$$

where $n_i$ is the number of documents in corpus where word $i$ appears.

In [11]:
idfs=np.log((1+C)/(1+word_counts))+1
idfs.head()

,count
word,
batteri,6.339539
raytheon,6.745004
intimid,6.521861
commentari,5.779923
issu,2.287754


<div class="alert alert-block alert-success"> We set up a few documents for comparison</div>

[HINT] Code below assumes that the variable `documents`  is the list of documents you read in problem 1.1

In [12]:
# document indexes we will use for comparison
document1=0 
document2=1
document3=105

# document filenames
filename1=documents["filename"][document1]
filename2=documents["filename"][document2]
filename3=documents["filename"][document3] # this will be from a different author

<div class="alert alert-block alert-info"> Problem 1.6 </div>
    Compute the word counts for `documents1`,`document2` and `document3`, using the `text_2_count` function defined at the beginning of the notebook.

In [13]:
count1=text_2_counts(filename1,stop_words)
count2=text_2_counts(filename2,stop_words)
count3=text_2_counts(filename3,stop_words)

The function below computes the normalized product of  `tfidf`  vectors.
Where the `tfidf` vector is defined as follows
$$
    w_{k} = \textrm{idf_k} * c_{k}
$$
where  $c_{k}$ is the number of times that word $k$ appears in document.

In [14]:
def product_tfidf(count1,count2,idfs):
    sum1=0.0
    sum_cross=0.0
    for key in count1:
        if key not in idfs.index:
            idf=0
            print(f"key {key} not found")
        else:
            idf=idfs.loc[key]["count"]
        w1=idf*count1[key]
        w2=idf*count2[key]
        sum1+=(w1)**2
        sum_cross+=w1*w2
    sum2=0.0
    for key in count2:
        if key not in idfs.index:
            idf=0
            print(f"key {key} not found")
        else:
            idf=idfs.loc[key]["count"]
        w2=idf*count2[key]
        sum2+=w2**2
    return sum_cross/np.sqrt(sum1*sum2)

It seems unlikely that 20 occurrences of a term in a document truly carry $20\times$ the significance of a single occurrence. And alternative (see the [Information Retrieval book](https://nlp.stanford.edu/IR-book/html/htmledition/sublinear-tf-scaling-1.html)) is to use a function
to *tamper* the growth of the word counts.

<div class="alert alert-block alert-info"> Problem 1.6 </div>
Create a function named `sublinear_product_tfidf`.
It should compute the normalized product of `tfidf` vectors as above but using a **`sublinear`** measure of  the word counts, defined as:
\begin{align}
    w_k  &= idf_k * (1+\log c_k)  &\textrm{if}\,\, c_k &>0 \\
    w_k  &= 0                    &\textrm{if}\,\, c_k &=0 \\
\end{align}
where $c_k$ is the raw word count for word $k$.

[HINT] Probably easiest to copy and modify slightly the function  `product_idf` above

In [15]:
def sublinear(c):
    if c>0:
        return 1+np.log(c)
    else:
        return 0

# function is identical to product_tfidf except for the three lines with comments.
def sublinear_product_tfidf(count1,count2,idfs):
    sum1=0.0
    sum_cross=0.0
    for key in count1:
        if key not in idfs.index:
            idf=0
            print(f"key {key} not found")
        else:
            idf=idfs.loc[key]["count"]
        w1=idf*sublinear(count1[key])     # change here!      
        w2=idf*sublinear(count2[key])     # change here!
        sum1+=(w1)**2
        sum_cross+=w1*w2
    sum2=0.0
    for key in count2:
        if key not in idfs.index:
            idf=0
            print(f"key {key} not found")
        else:
            idf=idfs.loc[key]["count"]
        w2=idf*sublinear(count2[key])   # change here!
        sum2+=w2**2
    return sum_cross/np.sqrt(sum1*sum2)

<div class="alert alert-block alert-info"> Problem 1.7 </div>
Compute the sublinear normalized product (similarity) for `document1` with itself, verify that the product is 1

In [16]:
sublinear_product_tfidf(count1,count1,idfs)

1.0

<div class="alert alert-block alert-info"> Problem 1.8 </div>
Compute the sublinear normalized products between 
1. `document1` and `document2`
2. `document1` and `document3`
3. `document2` and `document3`

In [17]:
sublinear_product_tfidf(count1,count2,idfs),sublinear_product_tfidf(count1,count3,idfs),sublinear_product_tfidf(count2,count3,idfs)

(0.11227509153067713, 0.041620049051990064, 0.034038033058980455)

## Comparison to  `sklearn`

<div class="alert alert-block alert-info"> Problem 2.1 </div>
store the value of the function `text.stem_tokenizer` from the module `text.py` into variable named `tokenizer`.

In [18]:
tokenizer=text.stem_tokenizer

<div class="alert alert-block alert-info"> Problem 2.2 </div>

set up  an instance of [`sklearn.TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)  so that it generates `tfidf` vectors using sublinear growth.

[Hint] 
1. Read carefully the  long list of options on the constructor of `TfidfVectorizer`
2. Do not forget to set the `input`, `tokenizer` and `stop_word` arguments.
    

In [19]:
tfidfVectorizer=TfidfVectorizer(input="filename",
                                tokenizer=tokenizer,
                                stop_words=stop_words,
                                sublinear_tf=True)

<div class="alert alert-block alert-info"> Problem 2.3 </div>
Generate the matrix $X$ of `tfidf` representations for each document in our corpus (this may take a bit of time)

In [20]:
X=tfidfVectorizer.fit_transform(documents["filename"])

<div class="alert alert-block alert-info"> Problem 2.4 </div>
Compute the dot product between `document1` and `document2` using their vector (`X`) representation. 

Compare to the result produced by the `sublinear_product_tfidf`
function you just wrote. They should be nearly identical.

In [21]:
np.dot(X[document1],X[document2].T)[0,0],sublinear_product_tfidf(count1,count2,idfs)

(0.11227509153067713, 0.11227509153067713)

### Saving Trained models for Reuse

<div class="alert alert-block alert-info"> Problem 3.1 </div>
In the data directory `data_dir`:
1. Save vectorizer to a `pickle` called "tfidf_sublinear_vectorizer.p"
2. Save sublinear `tfidf1` features to a file called "tfidf_sublinear_features.p"

In [22]:
tfidf_vectorizer_filename=   data_dir+"/tfidf_sublinear_vectorizer.p"
tfidf_features_filename=     data_dir+"/tfidf_sublinear_features.p"


pickle.dump(tfidfVectorizer, open( tfidf_vectorizer_filename, "wb" ) )
pickle.dump(X,              open( tfidf_features_filename, "wb" ) )


<div class="alert alert-block alert-info"> Problem 3.2 </div>
Make sure you can read those files again

In [23]:
tfidfVectorizer2=pickle.load( open(tfidf_vectorizer_filename, "rb" ))
X2=pickle.load(open(tfidf_features_filename, "rb" ) )
tfidfVectorizer2,X2

(TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='filename',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',...', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '``', "''"],
         strip_accents=None, sublinear_tf=True,
         token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=<function stem_tokenizer at 0x7fa202f41598>,
         use_idf=True, vocabulary=None),
 <2500x28060 sparse matrix of type '<class 'numpy.float64'>'
 	with 496636 stored elements in Compress